# Generation 테스트 코드

In [ ]:
!pip install transformers==2.11.0
!pip install sentencepiece

## 체크포인트, sentencepice 모델 다운로드

In [3]:
from google.colab import drive
drive.mount('/content/drive')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3250    0  3250    0     0   9420      0 --:--:-- --:--:-- --:--:--  9420
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1342      0 --:--:-- --:--:-- --:--:--  1342
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  485M    0  485M    0     0   155M      0 --:--:--  0:00:03 --:--:--  232M


In [4]:
%%bash
FILEID=19t6_Cn6qPM7HEq23zbeMQdeKtqGcEz73
file=kogpt2_news_wiki_ko_cased_818bfa919d.spiece

URL="https://drive.google.com/uc?export=download&id=$id"

wget --no-check-certificate "https://docs.google.com/uc?export=download&id=$FILEID" -O $file

--2020-07-18 17:08:10--  https://docs.google.com/uc?export=download&id=19t6_Cn6qPM7HEq23zbeMQdeKtqGcEz73
Resolving docs.google.com (docs.google.com)... 108.177.97.139, 108.177.97.101, 108.177.97.113, ...
Connecting to docs.google.com (docs.google.com)|108.177.97.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-04-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/260i35dccs9dl5e8vgej9u82446q8g6u/1595092050000/18111041224098424528/*/19t6_Cn6qPM7HEq23zbeMQdeKtqGcEz73?e=download [following]
--2020-07-18 17:08:11--  https://doc-00-04-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/260i35dccs9dl5e8vgej9u82446q8g6u/1595092050000/18111041224098424528/*/19t6_Cn6qPM7HEq23zbeMQdeKtqGcEz73?e=download
Resolving doc-00-04-docs.googleusercontent.com (doc-00-04-docs.googleusercontent.com)... 108.177.97.132, 2404:6800:4008:c00::84
Connecting to doc-00-04-docs.googleusercontent.com (doc-00-04-d

In [5]:
!ls -al

total 498804
drwxr-xr-x 1 root root      4096 Jul 18 17:08 .
drwxr-xr-x 1 root root      4096 Jul 18 17:02 ..
drwxr-xr-x 1 root root      4096 Jul 15 16:11 .config
-rw-r--r-- 1 root root 509587453 Jul 18 17:08 every_gpt.pt
-rw-r--r-- 1 root root   1170261 Jul 18 17:08 kogpt2_news_wiki_ko_cased_818bfa919d.spiece
drwxr-xr-x 1 root root      4096 Jul 10 16:29 sample_data


In [6]:
import torch
import sentencepiece
import transformers
from transformers import GPT2Config, GPT2LMHeadModel

In [7]:
model_file="/content/drive/My Drive/experiment/raw/every_gpt.pt"
tok_path = "kogpt2_news_wiki_ko_cased_818bfa919d.spiece"
kogpt2_config = {
    "initializer_range": 0.02,
    "layer_norm_epsilon": 1e-05,
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "n_positions": 1024,
    "vocab_size": 50000,
    "activation_function": "gelu"
}

In [8]:
tokenizer = sentencepiece.SentencePieceProcessor()
tokenizer.load(tok_path)

True

In [9]:
model = GPT2LMHeadModel.from_pretrained(pretrained_model_name_or_path=None,
                                    config=GPT2Config.from_dict(kogpt2_config),
                                    state_dict=torch.load(model_file))
model = model.cpu()

## Generation 방식 설정
* from [KoGPT2-FineTuning](https://github.com/gyunggyung/KoGPT2-FineTuning/tree/master/kogpt2/model)

In [44]:
def top_k_logits(logits, k):
    if k == 0:
        return logits
    values, _ = torch.topk(logits, k)
    min_values = values[:, -1]
    return torch.where(logits < min_values, torch.ones_like(logits, dtype=logits.dtype) * -1e10, logits)

def top_p_logits(logits, top_p=0.0, filter_value=-float('Inf')):
    """Nucleus sampling"""
    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs >= top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[:, indices_to_remove] = filter_value
    return logits

In [76]:
from collections import OrderedDict

text = "\uCEF4\uD559" #@param {type:"string"}
category = "\uC22D\uC2E4\uB300 \uC5D0\uD0C0" #@param ["\uC22D\uC2E4\uB300 \uC5D0\uD0C0", "\uBAA8\uB450\uC758 \uC5F0\uC560", "\uB300\uD559\uC0DD \uC7A1\uB2F4\uBC29"]

category_map = {
    "모두의 연애": "<unused3>",
    "숭실대 에타": "<unused5>",
    "대학생 잡담방": "<unused4>"
}

special_token_map = OrderedDict()

ids = tokenizer.encode_as_ids(text)
encoded = tokenizer.encode_as_pieces(text)

category_id = tokenizer.piece_to_id(category_map[category])
ids = [category_id] + ids

In [77]:
print(ids)
print(encoded)

[11, 2111, 47564]
['▁컴', '학']


In [78]:
from torch.nn import functional as F
duplicate_count = 0
duplicate_threshold = 10
for i in range(0, 512):
  input_ids = torch.tensor(ids).unsqueeze(0)
  pred = model(input_ids)[0]
  logits = pred[:, -1, :]
  logits = top_p_logits(logits, 0.3)
  log_probs = F.softmax(logits, dim=-1)
  prev = torch.multinomial(log_probs, num_samples=1)
  gen = prev[0].tolist()
  if gen[0] == tokenizer.eos_id():
      break
  duplicate_count = duplicate_count + 1 if ids[-1] == gen[0] else 0
  if duplicate_count > duplicate_threshold:
    break
  ids += gen
tokenizer.decode_ids(ids)

'<unused5> 컴학 컴퓨터수학2 계절학기 들으면 계절학기 때 들을 수 있나요?'

In [63]:
print('| Category : {}'.format(category))
print('| Conditioned : {}'.format(text))
print('| Generated : \n{}'.format(tokenizer.decode_ids(ids[1:])).replace('<unused2>', '\n').replace('<unused0>', 'https://...'))

| Category : 숭실대 에타
| Conditioned : 김
| Generated : 
김삼환 이사장 퇴진요구서 냈는데 읽어보니까 총학이 서명하면 끝인거같은데 어떻게 생각해?
